<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FUnet-scar/Processor_PerformanceTester_Scar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**

In [1]:
# Image processing
import cv2
import numpy as np

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os
import time
import math

## Mount google drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd /content/gdrive/MyDrive/Models/GAN_Scar
!ls -al

/content/gdrive/MyDrive/Models/GAN_Scar
total 205
drwx------ 2 root root  4096 Sep  8 13:49  Dataset
-rw------- 1 root root 22891 Sep  1 12:04  Image_segmentation-Scar.ipynb
drwx------ 3 root root  4096 Aug 17 05:49  Log
-rw------- 1 root root 63460 Sep  8 14:05  Processor_PerformanceTester-Scar.ipynb
drwx------ 4 root root  4096 Aug 16 08:39  Raw_Dataset
-rw------- 1 root root 16167 Sep  1 12:13  RawDataset_Processor-Scar.ipynb
drwx------ 2 root root  4096 Aug 23 14:47  result
-rw------- 1 root root 39995 Aug 15 11:40 'UNet architecture.PNG'
-rw------- 1 root root 49822 Sep  8 14:00  Unet-Scar.ipynb


# **Grobal variable**

In [4]:
# Path
MODEL_PATH = "/content/gdrive/MyDrive/Models/GAN_Scar"

RAW_TRAIN_SET_PATH = "/Raw_Dataset/train"
RAW_TEST_SET_PATH = "/Raw_Dataset/test"

DATASET_PATH = "/Dataset"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

# Image preprocess
NORM_INPUT_W_SIZE = 512
NORM_INPUT_H_SIZE = 384

# **Function**

In [5]:
def imshow_waitkey_enter(image):
    cv2_imshow(image)

    time.sleep(0.5)
    
    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

# **Make dir**

In [6]:
# Create processed dataset dir
train_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'train')
val_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'val')
test_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    os.makedirs(val_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# 사용 모델

## UNet

**마지막 Output 채널 2 → 1 변경**

<img src = "https://drive.google.com/uc?id=14CzAAaKv5v7pVfvugBRbD1xI4IuhmoyT"  width = 640>


<br>
<br>
<br>

# ***데이터셋 학습 비교 실험***


---

<br>

***공통사항***
* 원본 이미지 크기 = 244 * 244
* epoch = 40
* batch = 8
> UNet batch size = 4 <br>
> Wound dataset batch size = 8
* lr = 1e-3
* Train:Val = 8:2

<br>

***주의사항***
* 이미지의 사이즈는 정규화 되어야 함

<br>

***참고사항***
* CNN 대표 모델들의 네트워크 입력 사이즈는 224 * 224
* 이미지의 사이즈가 큰 경우에, Overlap-Tite(down sampling) 전략으로 한 이미지를 나눠서 학습시킨다.
* Polar Res UNet++ 의 input/output 사이즈는 [(384*512) == (y, x)](https://github.com/marinbenc/medical-polar-training/blob/main/datasets/lesion/make_dataset.py#L62)

<br>

---




<br>
<br>
<br>

## [대조군](https://www.nature.com/articles/s41598-020-78799-w)
> [Train code](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/train.py)
> <br>
> [Data generator](https://github.com/uwm-bigdata/wound-segmentation/blob/de3b9c00974b065b2062fd5cd2efe670da1f8f51/utils/io/data.py#L10)
> 1. 컬러 이미지(3채널)이 input으로 주어진다.
> 2. Raw dataset에 대한 전처리 없이, load 후 model에 들어가기 전에 Normalize

```
model = Deeplabv3(input_shape=(input_dim_x, input_dim_y, 3), classes=1)
```

<br>

***ISIC 2018 1위 모델, Polar Res UNet++ Dice score***
> Dice score = 0.925 (Dice loss = 0.075)
> <br>
> [Input size (y, x) = (384, 512)](https://github.com/marinbenc/medical-polar-training/blob/ecda91650279cd8794080a8ee19c1183faa8d022/datasets/lesion/make_dataset.py#L62)

### Performance



```
Input size =  torch.Size([8, 1, 224, 224])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.1887
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.1890
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.1885
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.1872
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.1864

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.1605
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.1673
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.1699
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.1710
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.1736
```



### Arrange dataset
> Not change anything about input image(scar)

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)

<br>
<br>
<br>

## 실험군

### 1

*적용사항*
1. Lab space를 이용한 Contrast 조정
2. BGR의 Green channel 사용

#### Performance

```
Input size =  torch.Size([8, 1, 224, 224])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2303
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2291
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2298
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2293
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.2305

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.2455
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.2397
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.2367
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.2480
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.2476
```

#### Process dataset

##### Train

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

##### Test

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)

### 2

*적용사항*
1. 불필요 Black boundary 제거
2. Lab space를 이용한 CLAHE Contrast 조정
3. BGR의 Green channel 사용
4. Resize input/output to 384*512 (y, x)
> UNet 아키텍쳐 입력 이미지 사이즈

#### Performance

```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.3232
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.3245
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.3232
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.3228
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.3211

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.3212
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.3204
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.3244
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.3239
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.3235
```

#### Process dataset

##### Train

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize image
    green_image = cv2.resize(green_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)
    label_image = cv2.resize(label_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize image
    green_image = cv2.resize(green_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)
    label_image = cv2.resize(label_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

##### Test

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize image
    green_image = cv2.resize(green_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)
    label_image = cv2.resize(label_image, 
                             (NORM_INPUT_W_SIZE, NORM_INPUT_H_SIZE),
                             interpolation=cv2.INTER_CUBIC)

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)

### 3 (학습 중)

*적용사항*
1. Black boundary 모두 제거
2. Lab space를 이용한 CLAHE Contrast 조정
3. BGR의 Green channel 사용
4. Bordering(cv2.copymakeborder CONSTANT) input/output to 384*512 (y, x)
> UNet 아키텍쳐 입력 이미지 사이즈

##### Performance

```
Input size =  torch.Size([8, 1, 384, 512])

TRAIN || EPOCH 0020 | BATCH  079 /  083 | DICE LOSS  0.2362
TRAIN || EPOCH 0020 | BATCH  080 /  083 | DICE LOSS  0.2352
TRAIN || EPOCH 0020 | BATCH  081 /  083 | DICE LOSS  0.2347
TRAIN || EPOCH 0020 | BATCH  082 /  083 | DICE LOSS  0.2344
TRAIN || EPOCH 0020 | BATCH  083 /  083 | DICE LOSS  0.2367

VALID || EPOCH 0020 | BATCH  017 /  021 | DICE_LOSS  0.3224
VALID || EPOCH 0020 | BATCH  018 /  021 | DICE_LOSS  0.3294
VALID || EPOCH 0020 | BATCH  019 /  021 | DICE_LOSS  0.3393
VALID || EPOCH 0020 | BATCH  020 /  021 | DICE_LOSS  0.3494
VALID || EPOCH 0020 | BATCH  021 /  021 | DICE_LOSS  0.3562

```

#### Process dataset

##### Train

In [7]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

##### Test

In [8]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_H_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_H_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_W_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_W_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)

### 4

*적용사항*
1. Black boundary 모두 제거
2. Lab space를 이용한 CLAHE Contrast 조정
3. BGR의 Green channel 사용
4. Resize input, label image width 또는 heigth 384 (비율 그대로)
5. Bordering(cv2.copymakeborder CONSTANT) input/output to 384*512 (y, x)
> UNet 아키텍쳐 입력 이미지 사이즈

#### Performance

```
Input size =  torch.Size([8, 1, 572, 572])


```

#### Process dataset

##### Train

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize images
    origin_scale = 1.0
    iters = 20
    for coefficient in range(1, iters + 1):
        resize_scale = origin_scale + (coefficient / 10)

        resize_w = width * resize_scale
        resize_h = height * resize_scale

        # Resizing
        if coefficient == iters or (resize_w == NORM_INPUT_W_SIZE or resize_h == NORM_INPUT_H_SIZE):
            green_image = cv2.resize(green_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
            label_image = cv2.resize(label_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
            break
        elif resize_w > NORM_INPUT_W_SIZE or resize_h > NORM_INPUT_H_SIZE:
            if coefficient - 1 != 0:
                resize_scale = resize_scale - ((coefficient - 1) / 10)
            else:
                resize_scale = origin_scale

            green_image = cv2.resize(green_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
            label_image = cv2.resize(label_image, 
                             (0, 0),
                             fx=resize_scale,
                             fy=resize_scale,
                             interpolation=cv2.INTER_CUBIC)
            break

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

##### Test

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)

    # Split each BGR
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)
    
    # resize

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    green_image = cv2.copyMakeBorder(green_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

### 5

*적용사항*
1. Black boundary 모두 제거
2. Lab space를 이용한 CLAHE Contrast 조정
3. K-Means(k=3)를 이용한 이미지 분류
4. Resize input, label image width 또는 heigth 384 (비율 그대로)
5. Bordering(cv2.copymakeborder CONSTANT) input/output to 384*512 (y, x)
> UNet 아키텍쳐 입력 이미지 사이즈

#### Performance

```
Input size =  torch.Size([8, 1, 572, 572])


```

#### Process dataset

##### Train

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Clustering use K-means
    cluster_data = enhanced_image.reshape((-1, 3)).astype(np.float32)
    k = 3
    criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0) # 최대 10번 반복하고 1픽셀 이하로 움직이면 종료
    ret, c_label, center = cv2.kmeans(cluster_data, 
                                    k, 
                                    None, 
                                    criteria,
                                    10,
                                    cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)
    kmeans_image = center[c_label.flatten()]
    kmeans_image = kmeans_image.reshape((enhanced_image.shape))
    kmeans_image = cv2.cvtColor(kmeans_image, cv2.COLOR_BGR2GRAY)

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Clustering use K-means
    cluster_data = enhanced_image.reshape((-1, 3)).astype(np.float32)
    k = 3
    criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0) # 최대 10번 반복하고 1픽셀 이하로 움직이면 종료
    ret, c_label, center = cv2.kmeans(cluster_data, 
                                    k, 
                                    None, 
                                    criteria,
                                    10,
                                    cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)
    kmeans_image = center[c_label.flatten()]
    kmeans_image = kmeans_image.reshape((enhanced_image.shape))
    kmeans_image = cv2.cvtColor(kmeans_image, cv2.COLOR_BGR2GRAY)

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{idx:03d}.png'), label_image)

##### Test

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Clustering use K-means
    cluster_data = enhanced_image.reshape((-1, 3)).astype(np.float32)
    k = 3
    criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0) # 최대 10번 반복하고 1픽셀 이하로 움직이면 종료
    ret, c_label, center = cv2.kmeans(cluster_data, 
                                    k, 
                                    None, 
                                    criteria,
                                    10,
                                    cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)
    kmeans_image = center[c_label.flatten()]
    kmeans_image = kmeans_image.reshape((enhanced_image.shape))
    kmeans_image = cv2.cvtColor(kmeans_image, cv2.COLOR_BGR2GRAY)

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)